In [1]:
import json
import os
import traceback
import re
import requests
import zipfile,io
import glob
import numpy as np
from shutil import copy2,copytree,rmtree
import xml.etree.ElementTree as ET
from bs4 import BeautifulSoup
import openbabel
import time
import random
import sys
import tempfile

#HTMD things
import htmd
from htmd.ui import *
from moleculekit.tools.sequencestructuralalignment import sequenceStructureAlignment
from htmd.protocols.equilibration_v2 import Equilibration
from htmd.protocols.production_v6 import Production
from htmd.builder.builder import removeLipidsInProtein, tileMembrane, minimalRotation,removeAtomsInHull
from moleculekit.util import rotationMatrix, sequenceID, opm
from htmd.builder.charmm import _recoverProtonations
from htmd.config import config

# IMPORTANT!!!! vmd needs to be installed for this pipeline to run properly
config(viewer='vmd')

/soft/EB_repo/devel/programs/foss/2016b/Anaconda3/5.0.1/lib/python3.6/site-packages/htmd/molecule/util.py:666: NumbaPerformanceWarning: np.dot() is faster on contiguous arrays, called on (array(float32, 2d, A), array(float32, 2d, A))
  covariance = np.dot(P.T, Q)
/soft/EB_repo/devel/programs/foss/2016b/Anaconda3/5.0.1/lib/python3.6/site-packages/htmd/molecule/util.py:704: NumbaPerformanceWarning: np.dot() is faster on contiguous arrays, called on (array(float32, 2d, A), array(float32, 2d, A))
  all1 = np.dot(all1, rot.T)


ffevaluate module is in beta version

Please cite HTMD: Doerr et al.(2016)JCTC,12,1845. https://dx.doi.org/10.1021/acs.jctc.6b00049

HTMD Documentation at: https://www.htmd.org/docs/latest/



2020-09-09 09:20:30,578 - binstar - INFO - Using Anaconda API: https://api.anaconda.org
/soft/EB_repo/devel/programs/foss/2016b/Anaconda3/5.0.1/lib/python3.6/site-packages/pyemma/util/log.py:65: YAMLLoadWarning: calling yaml.load() without Loader=... is deprecated, as the default Loader is unsafe. Please read https://msg.pyyaml.org/load for full details.
  D = yaml.load(f2)


New devel HTMD version (1.22.7 python[3.7,<3.8.0a0,3.6,<3.7.0a0]) is available. You are currently on (1.13.10).There are several methods to update:    - Create a new conda env. using `conda create -n htmd1.22.7 htmd=1.22.7 -c acellera -c psi4`    - Create a brand new conda installation and run `conda install htmd -c acellera -c psi4`    - Run: `conda update htmd -c acellera -c psi4` (NOT RECOMMENDED)



In [2]:
####################
## Initial variables
####################

# PDB codes of the GPCRs to be simulated. 
# If no codes are provided, all avalible structures in GPCRdb will be used (except the ones already simulated)
noncomplex = {"6TPK", "6S0Q", "6V9S", "6WJC", "6W25", "6OBA", "6LW5", "6KP6", "6LUQ", "6LI1", "6LI0", "6LI2", "6KPC", "6LRY", "6KNM", "6TOT", "6TOS", "6TO7", "6TP6", "6TQ7", "6TOD", "6TP3", "6TQ4", "6TQ9", "6TP4", "6TQ6", "6TPN", "6TPG", "6TPJ", "6OL9", "6RZ6", "6RZ9", "6RZ7", "6RZ8", "6PT2", "6PT3", "6KUX", "6KUY", "6KUW", "6IQL", "6PS7", "6PS0", "6PS3", "6PS5", "6PS1", "6PS4", "6PRZ", "6PS2", "6PS6", "6KK1", "6KK7", "6KJV", "6PS8", "6JZH", "6RZ4", "6RZ5", "6KQI", "6QZH", "6K1Q", "6GT3", "6MH8", "6ME2", "6ME3", "6ME4", "6ME5", "6ME6", "6ME7", "6ME8", "6ME9", "6J21", "6J20", "6A94", "6A93", "5ZTY", "6HLP", "6HLO", "6HLL", "6GPX", "6GPS", "6IIV", "6IIU", "6E59", "6M9T", "6AK3", "5ZHP", "5ZKC", "5ZK3", "5YC8", "5ZK8", "5ZKB", "6IGK", "6IGL", "6FJ3", "6AKX", "6AKY", "6D27", "6D26", "6DRX", "6DS0", "6DRY", "6DRZ", "6BD4", "5ZKQ", "5ZKP", "6C1R", "6C1Q", "6D32", "6D35", "5KW2", "5ZBH", "5ZBQ", "6FK7", "6FKA", "6FKD", "6FK6", "6FK9", "6FKC", "6FK8", "6FKB", "6CM4", "6FFI", "6FFH", "5WF5", "5WF6", "6BQH", "6BQG", "5V54", "5OLH", "5OLZ", "5OLO", "5OM1", "5OLG", "5OLV", "5OM4", "5YQZ", "6AQF", "5O9H", "5X33", "5VRA", "5WS3", "5WQC", "5WIV", "5WIU", "5NM4", "5NM2", "5NLX", "5X7D", "5X93", "5XPR", "5XSZ", "5N2S", "5MZP", "5MZJ", "5N2R", "5XRA", "5XR8", "5UIW", "5NX2", "5TZY", "5TZR", "5JTB", "5VBL", "5UVI", "5VEW", "5V56", "5V57", "5VEX", "5NDD", "5NDZ", "5UNH", "5UNF", "5UNG", "5TE3", "5TE5", "5UEN", "5UIG", "5TVN", "5T04", "5T1A", "5LWE", "5U09", "5TGZ", "5K2C", "5K2B", "5K2A", "5K2D", "5GLI", "5GLH", "5D6L", "5DYS", "5L7D", "5L7I", "5IU7", "5IUB", "5IU8", "5IU4", "5IUA", "4Z9G", "5EE7", "5DSG", "5CXV", "4ZJ8", "4ZJC", "5F8U", "5DHG", "5DHH", "4ZUD", "5A8E", "5CGC", "5CGD", "4XEE", "4XES", "4Z35", "4Z34", "4Z36", "4YAY", "4UHR", "4XNW", "4XNV", "4XT3", "4RWS", "4RWD", "4S0V", "4U15", "4U16", "4QIM", "4QIN", "4PHU", "4OO9", "4PXZ", "4PY0", "4BVN", "4NTJ", "4OR2", "4O9R", "4BUO", "4N4W", "4N6H", "4NC3", "4MBS", "4L6R", "4K5Y", "4JKV", "3ZPR", "3ZPQ", "4IAQ", "4IAR", "4IB4", "4GPO", "3VW7", "4GBR", "4GRV", "4EIY", "4AMJ", "4AMI", "4EJ4", "4EA3", "3UZA", "3UZC", "4DJH", "4DKL", "3V2Y", "3UON", "3REY", "3RFM", "3RZE", "2YCZ", "2YCW", "2YDV", "2YDO", "3QAK", "2Y04", "2Y00", "2Y03", "2Y02", "3PDS", "3PBL", "3ODU", "3OE0", "3NY9", "3NY8", "3NYA", "3D4S", "2Z73", "2RH1", "1U19", "1GZM"}
noncomplex_nonGPCRmd = {'6RZ8', '6FFI', '6KJV', '6TQ6', '6DRY', '5WIV', '5V57', '6LI2', '6MH8', '5MZJ', '5KW2', '5X33', '5LWE', '5ZHP', '6PS6', '6FJ3', '6D35', '4NTJ', '6A94', '6AK3', '5UVI', '6FFH', '5WIU', '6KQI', '5XR8', '6RZ9', '5VEX', '6FKD', '6LW5', '6FKA', '6PS7', '6M9T', '6KNM', '6ME6', '6GPX', '5XSZ', '5K2A', '5YQZ', '6PS0', '5K2C', '6ME7', '2Z73', '6D32', '6RZ4', '5YC8', '5T1A', '6AKY', '4N4W', '5WS3', '6LRY', '6J21', '6BQG', '5TE5', '4O9R', '6FK6', '1GZM', '6DRX', '6TQ7', '5NDZ', '6IGL', '5VRA', '5ZBQ', '6TPN', '5ZTY', '6DS0', '5NLX', '5X93', '6HLP', '5OLG', '5OM4', '5O9H', '5XPR', '6A93', '5GLI', '6OBA', '6D27', '6HLO', '6RZ5', '5TE3', '6IIV', '6KP6', '6TQ4', '6TPJ', '5V56', '5UNG', '6FK9', '6PS8', '2YCZ', '6JZH', '5XRA', '5ZKB', '4JKV', '5MZP', '5K2D', '5OLV', '5X7D', '5T04', '5OLO', '5DYS', '6FK7', '6J20', '6K1Q', '5EE7', '6D26', '6PS1', '6DRZ', '4GBR', '6FKC', '6TP6', '6LUQ', '5ZKQ', '5UNH', '6PRZ', '5UNF', '5NX2', '6LI0', '5NM2', '5D6L', '6AQF', '6KK1', '6C1Q', '4Z9G', '6LI1', '5WF5', '6FKB', '6BQH', '6QZH', '5OLH', '4XT3', '5ZK3', '5VEW', '4XES', '6HLL', '6TOT', '5N2R', '5WF6', '5K2B', '6TPG', '6PS4', '5F8U', '6ME4', '6GT3', '6RZ6', '6ME8', '6ME2', '5VBL', '5N2S', '5ZBH', '6PS3', '6IIU', '4NC3', '5JTB', '5ZKP', '6TQ9', '6GPS', '6TP3', '6RZ7', '6ME3', '5ZK8', '6C1R', '5V54', '5UIG', '5TVN', '5OM1', '6AKX', '5ZKC', '5NM4', '5TZY', '6TOD', '4GPO', '6TO7', '6CM4', '6TOS', '6PS5', '6ME9', '5NDD', '6KPC', '6KK7', '4BUO', '6OL9', '6PS2', '6FK8', '6ME5', '6TP4', '4QIM', '6IGK', '5WQC', '5UIW', '5TZR', '5OLZ', '4EJ4', '6E59'}
pdb_set = noncomplex_nonGPCRmd

# Get current GPCRdb data into a Json
gpcrdb_data = requests.get('http://gpcrdb.org/services/structure/').json()

# Path to slurm queing system binaries
# In our case, Ismael designed a bunch of small bash scripts (fake_slurm) which do ssh to Hydra and execute slurm there
slurmpath = '/gpcr/users/daranda/doctorat/GPCR_simulations/fake_slurm/'
path= os.environ['PATH']
%env PATH=$path:$slurmpath

#Path to ACEMD in computation node and ACEMD license
acemd_path = "/opt/acellera/miniconda3/bin/acemd3"
acemd_license = "SG_LICENSE_FILE=28000@tolkien.prib.upf.edu"

# Other Paths
basepath = '/gpcr/users/daranda/doctorat/GPCR_simulations/'
resultspath = basepath + 'simulation_output/'
membranepdb = basepath + 'membrane/popc36_box_renumbered.pdb'
topparpath = basepath + 'toppar'#toppar= topology+parameters
ligandsdict_path = basepath + 'ligands.json'

# Parameters
username = 'paramoid'#ameboid
password = 'paramoid-123'#ameboid-123
new_pdb_chain = 'P'
membrane_lipid_segid = 'MEM'
coldist = 1.3 # Distance bellow which two atoms are considered to collide
buffer = 2.4 # Distance between solvation waters and the rest of the system
water_thickness = 20 # Size in Z-axis of the solvation water layers
membrane_distance = 20 # Distance between the pbc box and the rest of the system atoms, to be filled with membrane
water_margin = 4 # Distance in the Z-axis to be penetrated by the solvation box 
                 # to avoid the formation of a V O I D between the system and the solvation boxes
detergent_blacklist = ['OLA','OLB','PEG','GOL','BOG','OLC','P6G','P33','UNL','UNX','PLM','HTG',
                       '12P','LPP','PEF','2CV','SOG','TWT','PGE','SO4','STE','LMT','ACT','ACE',
                      'MHA','CIT','1PE','MPG','EPE','PG4','DGA','PO4','DMS','TAR','1WV','EDO',
                      'BU1','ACM','PG6','TLA','SCN','TCE','MES','EDT','POV','MLI','SIN','PGO',
                      'FLC','HTO','PGW']
glucids_blacklist = ['MAN','NAG','BGC','TRE','9NS','BMA','FUC']
    
# Dummy pdb: a pdb made of a sinlge non-existant DUM atom.
# It is used during removal of aromatic insertions by placing it on the middle of the ring and measuring distances  
dummypdb='./dummy.pdb'
try:
    dummymol = Molecule(dummypdb, validateElements=False)
except Exception as e:
    dummymol = Molecule(dummypdb)
dummy_sel = 'name DUM'

# Topologies filenames and paths
toposfilenames = ['top_all36_prot.rtf',
                  'top_all36_na.rtf',
                  'top_all36_lipid.rtf',
                  'top_all36_carb.rtf',
                  'top_all35_ethers.rtf',
                  'top_all36_cgenff.rtf']
topostreams = ['toppar_water_ions_1.rtf','toppar_ions_won.rtf']
streams_folder='stream_splits'
topos = [os.path.join(topparpath,file) for file in toposfilenames] + \
        [os.path.join(os.path.join(topparpath,streams_folder),file) for file in topostreams] 

# Parameters filenames and paths
paramsfilenames = ['par_all36m_prot.prm','par_all36_na.prm','par_all36_lipid.prm','par_all36_carb.prm',\
                  'par_all35_ethers.prm','par_all36_cgenff.prm']
paramsstreams = ['toppar_water_ions_1.inp','toppar_water_ions_2.inp']
params = [os.path.join(topparpath,file) for file in paramsfilenames] + \
         [os.path.join(os.path.join(topparpath,streams_folder),file) for file in paramsstreams]

env: PATH=/soft/EB_repo/devel/programs/foss/2016b/Anaconda3/5.0.1:/soft/EB_repo/devel/programs/foss/2016b/Anaconda3/5.0.1/bin:/soft/EB_repo/devel/programs/goolf/1.7.20/Python/3.6.2/bin:/soft/EB_repo/devel/programs/goolf/1.7.20/netCDF/4.4.1/bin:/soft/EB_repo/devel/programs/goolf/1.7.20/cURL/7.43.0/bin:/soft/EB_repo/devel/programs/goolf/1.7.20/HDF5/1.8.17/bin:/soft/EB_repo/devel/programs/goolf/1.7.20/Tk/8.6.4/bin:/soft/EB_repo/devel/programs/goolf/1.7.20/XZ/5.2.2/bin:/soft/EB_repo/devel/programs/goolf/1.7.20/SQLite/3.9.2/bin:/soft/EB_repo/devel/programs/goolf/1.7.20/Tcl/8.6.4/bin:/soft/EB_repo/devel/programs/goolf/1.7.20/libreadline/6.3/bin:/soft/EB_repo/devel/programs/goolf/1.7.20/ncurses/5.9/bin:/soft/EB_repo/devel/programs/goolf/1.7.20/bzip2/1.0.6/bin:/soft/EB_repo/devel/programs/goolf/1.7.20/FFTW/3.3.4/bin:/soft/EB_repo/devel/programs/goolf/1.7.20/OpenBLAS/0.2.13/bin:/soft/EB_repo/devel/programs/goolf/1.7.20/OpenMPI/1.8.4/bin:/soft/EB_repo/devel/programs/goolf/1.7.20/hwloc/1.10.1/bin

In [3]:
####################
## David's functions
####################
def ligands_by_system(ligandsdict):
    """
    Creates a json with all the systems codes and their ligand molecules 
    """
    with open('ligands_by_system_new.json', 'w') as ou:
        dasdict = {}
        for pdb in ligandsdict:
            dasdict[pdb] = list(ligandsdict[pdb].keys())
        json.dump(dasdict, ou, indent= 4)

def json_dict(path):
    """
    Converts json file to pyhton dict.
    """
    json_file=open(path)
    json_str = json_file.read()
    json_data = json.loads(json_str)
    return json_data

def get_GPCRdb_nonsimulated(gpcrdb_data):
    """
    Returns a list of PDB codes from the GPCRdb refined structures not yet simulated in GPCRmd
    """
    
    #Make set with the pdb codes of the structures in GPCRdb
    gpcrdb_pdbs = { struc['pdb_code'] for struc in gpcrdb_data }

    # Load a random name-to-dyn json from contactmaps
    # This Jsons contain a dictionary with the dynIDs and the full names of the GPCR simulated
    # This way I can get all the pdb codes of the GPCRs presents in GPCRmd
    response = requests.get('http://submission.gpcrmd.org/dynadb/files/Precomputed/get_contacts_files/contmaps_inputs/all/cmpl/lg/name_to_dyn_dict.json')
    soup = BeautifulSoup(response.text, 'html.parser')
    gpcrmd_sims = json.loads(str(soup))

    # Take the pdb code from each full name in the json, and store in set
    gpcrmd_pdbs = set()
    pdb_pat = re.compile('\((\w+).*\) \(.*\)$')# Take objects whatever is inside of the first parenthesis
    for sim in gpcrmd_sims:
        match_pdb = re.search(pdb_pat, sim[1])
        if match_pdb:
            gpcrmd_pdbs.add(match_pdb.group(1))

    # Get to-be-simulated GPCR pdbs. That is the ones that are in GPCRdb but not in GPCRmd
    not_simulated = gpcrdb_pdbs - gpcrmd_pdbs

    return not_simulated

def download_GPCRdb_structures(pdb_set, basepath):
    """
    Download (if they exist) the refined GPCRdb structures for the pdb codes in the pdb_set.
    PDB codes without a refined structure will be removed from pdb_set
    """
    pdb_set_nonrefined = set()
    set_length = len(pdb_set)
    i = 0
    for pdbcode in pdb_set:
        simdir = basepath+'/data_sim/'+pdbcode+'/'
        os.makedirs(simdir, exist_ok = True)
        i += 1
        
        print('Downloading %s structure (%d/%d)' % (pdbcode, i, set_length))
        # If files for this simulation already exists
        if glob(simdir+'*pdb'):
            print('Structure for %s already present. Skipping...' % pdbcode)
        else:
            response = requests.get('https://gpcrdb.org/structure/homology_models/'+pdbcode+'_refined/download_pdb', stream=True)
            if response.ok:
                zippy = zipfile.ZipFile(io.BytesIO(response.content))
                zippy.extractall(simdir)
            else:
                pdb_set_nonrefined.add(pdbcode)
                print('could not download %s refined structure. Skipping...' % (pdbcode))
    
    #Remove non-refined structrues
    pdb_set = pdb_set - pdb_set_nonrefined
    
    return pdb_set

def ligand_dictionary(pdb_set, ligandsdict_path):
    """
    Create dictionary with ligand names and ligand ResNames of each of the structures we need to simulate,
    and store the resutls in a json file
    """
    # Read existing ligands dictionary, or create it if it does not exists yet 
    if os.path.exists(ligandsdict_path):
        ligandsdict = json_dict(ligandsdict_path)
    else:
        ligandsdict = {}

    # Iterate over non-yet-simulated structures, and get their ligand information from rcsb (PDB's web api)    
    for pdb_code in pdb_set:
        #Do not repeat simulations
        if pdb_code in ligandsdict:
            continue
        else:
            ligandsdict[pdb_code] = {}
            response = requests.get('http://www.rcsb.org/pdb/rest/customReport.xml?pdbids='+pdb_code+'&customReportColumns=ligandId,ligandName')
            ligand_tree = ET.fromstring(response.content)
            for ligand in ligand_tree:
                ligandResname = ligand.find('dimEntity.ligandId').text
                ligandName = ligand.find('dimEntity.ligandName').text
                if ligandResname == 'null':
                    continue
                else:
                    ligandsdict[pdb_code][ligandResname] = ligandName

    with open(ligandsdict_path, 'w') as jsonfile:
        json.dump(ligandsdict, jsonfile, ensure_ascii=False, indent = 4)            
    
    # Create ligands set from previou dictionary
    ligandsset = { ligcode  for pdbcode in ligandsdict for ligcode in ligandsdict[pdbcode] }
        
    return(ligandsdict, ligandsset)

def extract_ligands(ligandsdict, basepath):
    """
    Extract ligands from the refined structure PDB and convert htem to a mol2 file
    """
    
    obConversion = openbabel.OBConversion()
    obConversion.SetInAndOutFormats("pdb", "mol2")

    # Iterate over ligands
    for system in ligandsdict:
    
        # Skip if structure of this system is not avalible
        syspdb_path_list = glob(str("%sdata_sim/%s/*pdb" % (basepath,system)))
        if len(syspdb_path_list) == 0:
            continue
        else:
            syspdb_path = syspdb_path_list[0]
        
        for ligcode in ligandsdict[system]:
        #for ligcode in ['TYS']:
            ligpath = basepath+"Ligands/"+ligcode+"/"
            #Skip if ligand has already been download
            if os.path.exists(ligpath):
                continue
            else:
                # Directory for ligands
                os.makedirs(ligpath, exist_ok=True)

                # Take ligand PDB from inside the GPCR system PDB
                syspdb_path = glob(str("%sdata_sim/%s/*pdb" % (basepath,system)))[0]
                ligpdb_path = ligpath + "ligand.pdb"
                ligpdb = open(ligpdb_path, 'w')
                atomnames = set()
                with open(syspdb_path, 'r') as syspdb:
                    for line in syspdb:
                        resname = line[17:20]
                        if resname == ligcode:
                            atomname = line[13:20]
                            if atomname not in atomnames:# If there are two or more molecules of one ligand, take only one 
                                ligpdb.write(line)
                                atomnames.add(atomname)
                ligpdb.close()

                #Convert SDF to mol2, and save mol2 in corresponding folder
                ligand_mol2 = openbabel.OBMol()
                ligand_mol2.AddHydrogens()
                obConversion.ReadFile(ligand_mol2, ligpdb_path)
                ligand_mol2.AddHydrogens()
                ligandmol_string = obConversion.WriteString(ligand_mol2)
                
                # Change name of molecule in mol2
                with open(ligpath+"ligand.mol2", 'w') as ligandmol_file:
                    ligandmol_file.write(ligandmol_string.replace(ligpdb_path, ligcode))
                    
def get_lig_toppar(ligandsset, basepath, username, password):
    """
    Get the topology-parameters string file from paramchem for the submited ligand PDB codes
    ALERT: paramchem only allows 100 submissions by month, so it may be possible that not all 
    parameters are obtained
    """
    
    #Get total number of ligands
    i = 0
    total_ligs = len(ligandsset)
    #Pattern to find non-HTMD-compatible lines
    lph_pat = re.compile('^ATOM.*LPH|LONEPAIR')
    
    # Iterate over ligands
    for ligcode in ligandsset:
        i += 1
        print('Getting toppar file for ligand %s (%d/%d)' % (ligcode, i, total_ligs))

        # topology-parametetrs file output
        topparfile_path = basepath+"Ligands/"+ligcode+"/toppar.str"
        # Errors and warnings file output
        erwar_path = basepath+"Ligands/"+ligcode+"/paramchem_stder.txt"
        #Open ligandfile to upload in paramchem
        ligfile = open(basepath+"Ligands/"+ligcode+"/ligand.mol2")
        myfile = {
                'filename': ligfile
        }
        myparam = {
                #'param_a': 'a' #Include parameters usually included in newer versions of CGenff (versions that we cant use)
                'param_b' : 'b' # Make paramchem determine bond order
                #'param_c': 'c'# Use CGenFF legacy v1.0, for HTMD is not yet prepared for newer Charmm versions             
        }

        # Omit ligand if its toppar file already exists
        if os.path.exists(topparfile_path):
            print('toppar for ligand %s already exists. Skipping...' % ligcode)
            continue
        else:
            # Define POST variables for login in Paramchem
            datalogin = {
                'usrName': username,
                'curPwd': password,
                'rndNo': str(random.randrange(100000000, 999999999)),
                'submitBtn': 'Submit',
            }
            # Open web session
            with requests.Session() as s:

                # Login into paramchem
                response_login = s.post('http://cgenff.umaryland.edu/userAccount/userWelcome.php', 
                           data=datalogin,
                           verify=False)

                # Submit our ligand molecule into paramchem
                response_upload = s.post('http://cgenff.umaryland.edu/initguess/processdata.php', 
                           files = myfile,
                           data = myparam,
                            )

                # Download Topology-parameters of our molecule file from paramchem, and store it.
                # But remember submissions in paramchem are limited weekly
                # Download also stderr, just in case
                match = re.search('<path>(\w+)</path>', response_upload.text)
                if match:
                    code = match.group(1)
                    response_ligfile = s.get('http://cgenff.umaryland.edu/initguess/filedownload.php?file='+code+'/ligand.str')
                    response_stder = s.get('https://cgenff.umaryland.edu/initguess/filedownload.php?file='+code+'/ligand.err')
                    with open(topparfile_path, 'wb') as topparfile:
                            topparfile.write(response_ligfile.content)
                    with open(erwar_path, 'wb') as erwar:
                            erwar.write(response_stder.content)                            
                else:
                    print('Your paramchem account has reached its weekly submission limit. Please, intrudce a new account or wait to the next monday to continue')            
                    return
                
                #Delete lines with LPH (new feature from CHARMM not tolerated by HTMD)
                with open(topparfile_path, "r") as f:
                    lines = f.readlines()
                with open(topparfile_path, "w") as f:
                    for line in lines:
                        if not re.match(lph_pat, line):
                            f.write(line)

def internal_waters(simdir, pdbcode, gpcrdb_data):
    """
    Place internal waters in GPCR structure using homolwat online tool
    """
    # Guess if molecule is active or not, and make Homolwat place (inactive) or not (active)
    # the 2.50 sodium ion
    sod = None
    agoset = {'Agonist', 'Agonist (partial)'}
    for entry in gpcrdb_data:
        if entry['pdb_code'] == pdbcode:
            ligands = entry['ligands']
            if len(ligands) == 0: # If apoform
                sod = "sod_yes"
            elif ligands[0]['function'] in agoset: 
                sod = "sod_no"
            else:
                sod = "sod_yes"
    
    # Check if there is already a watered structure here
    if glob(simdir+'*_HW.pdb'):
        print("Structure %s already has a watered version. Skipping..." % pdbcode)
        return (sod == 'sod_yes')
    else:
        print("Adding internal waters to structure")
        
    #Load pdb file
    pdbpath = glob(simdir+'*.pdb')[0]
    pdbfiles = {"file" : open(pdbpath, 'rb')} 
    
    # Open web session
    with requests.Session() as s:
        # Load our desired structure into homolwat, as a PDB file
        response_loadfile = s.post("https://alf06.uab.es/homolwat/run_HW",
                                   files = pdbfiles
        )

        # Get name and number of our query. Required for following steps
        soup = BeautifulSoup(response_loadfile.text,'html')
        query_num = soup.find('input',attrs={'name' : 'query_num'}).get('value')
        query_name = soup.find('input',attrs={'name' : 'query_name'}).get('value')
        
        # Analyze structure and place internal waters
        response_solvate = s.post("https://alf06.uab.es/homolwat/solvate",
                  data = {
                    "query_name": query_name,
                    "query_num": query_num,
                    "ch_rest": '[]',
                    "option_state": "inac",
                    "option_sodium": sod,
                    "option_dowser": "dow_no",
                    "p_ident": ""
                  }
        )
        # Download results in zip
        query_num_noapo = query_num.split("'")[1]
        query_name_nofilext = query_name.split(".")[0]
        response_download = s.post("https://alf06.uab.es/homolwat/download_file/",
              data = {
                    "query_num": query_num_noapo,
                    "query_name": query_name_nofilext
              })
        
        # Unzip and extract watered strucutre file
        req = response_download.request
        if response_download.ok:
            zippy = zipfile.ZipFile(io.BytesIO(response_download.content))
            zippy.extract(query_name_nofilext+"_HW.pdb", path=simdir)
            
            # This program sodiums are wrongly formated, so the name of the homolog
            # simulation occupies the charge column by error. Here i solve this
            if sod == 'sod_yes':
                in_hw = open(simdir + query_name_nofilext+"_HW.pdb", 'r')
                out_hw = open(simdir + query_name_nofilext+"_provo.pdb", 'w')
                for line in in_hw:
                    if (len(line) > 79) and (line[79] != " "):
                        linelist = list(line)
                        linelist[79] = " "
                        line = "".join(linelist)
                    out_hw.write(line)
                os.remove(simdir + query_name_nofilext+"_HW.pdb")
                os.rename(simdir + query_name_nofilext+"_provo.pdb", simdir + query_name_nofilext+"_HW.pdb")
        else:
            print("could not add internal waters to %s. Skipping..." % (pdbpath))
    return (sod == 'sod_yes')

def get_opm(pdbcode):
    """
    Download and load OPM molecule for this PDB code. 
    If there is no opm structure for this pdbcode, download the substitute structure OPM seems fit
    Return thickness and opm molecule
    """
    
    # Search pdb code in OPM database, and take thickness and opm_id
    searchlink = 'https://lomize-group-opm.herokuapp.com//primary_structures?search='+pdbcode+'&sort=&pageSize=100'
    response_dict=eval(requests.get(searchlink).content.decode('UTF-8')\
                       .replace('true', 'True')\
                       .replace('false','False')\
                       .replace('null','None'))
    thickness = response_dict['objects'][0]['thickness']
    opm_id = response_dict['objects'][0]['id']

    # Throught opm id, get PDB id of the reference structure in OPM for this pdbcode
    
    opmlink = 'https://lomize-group-opm.herokuapp.com//primary_structures/'+str(opm_id)
    response_dict=eval(requests.get(opmlink).content.decode('UTF-8')\
                       .replace('true', 'True')\
                       .replace('false','False')\
                       .replace('null','None'))
    new_pdbcode = response_dict['pdbid'].lower()
    
    # Download OPM pdb file, and save all lines not involving a DUM residue in a temp file 
    response_pdb = requests.get('https://opm-assets.storage.googleapis.com/pdb/'+new_pdbcode+'.pdb')
    line_list = (response_pdb.content.decode('UTF-8').split('\n'))
    tmpout = tempfile.NamedTemporaryFile(mode='w',suffix='.pdb')
    for line in line_list:
        if ' DUM ' not in line:
            tmpout.write(line+'\n')
    name = tmpout.name
    mol = Molecule(name)
    tmpout.close()
    
    return (thickness, mol)

def remove_artifacts(pdbcode, mol, ligdict, accepted_ligdict):
    """
    Remove any small molecules included in ligdict but not in accepted_ligdict.
    The intention is to remove unnecessary small molecules, like detergents 
    or ligands from removed parts of the protein
    """
    tofilter = ""
    for smalmol in ligdict[pdbcode]:
        if smalmol not in accepted_ligdict[pdbcode]:
            print('no '+smalmol)
            tofilter += smalmol + " "
    if tofilter:
        gpcrdb_mol.filter('not resname '+tofilter)
    return gpcrdb_mol
                            
#####################
## Ismael's Functions
#####################

def renumber_resid_vmd(mol,sel,by=3,start=1):
    tmpin = tempfile.NamedTemporaryFile(suffix='.pdb')
    mol.write(tmpin.name)
    viewer = getCurrentViewer(dispdev='text')
    viewer.send('set molid [mol new {%s}]' % tmpin.name)
    tmpin.close()
    tmpout = tempfile.NamedTemporaryFile(suffix='.pdb')
    
    # What a wierd way of deciding if "by_segid", "by_resname" or by both
    option_num = 2
    max_value = 2**option_num - 1
    if by > max_value:
        raise ValueError('Maximum value for "by" keyword is %d.' % max_value)
    if by < 1:
        raise ValueError('Minimum value for "by" keyword is "1".')
    bin_by = format(by,'0'+str(option_num)+'b')
    option_array = [bool(int(i)) for i in bin_by]
    by_segid = option_array[0]
    by_resname = option_array[1]
    
    if by_segid:
        segids = set(mol.get('segid',sel=sel))      
        for segid in segids:
            if by_resname:
                resnames = set(mol.get('resname',sel='(%s) and segid %s' % (sel,segid)))
                for resname in resnames:
                    lsel = '(%s) and (segid %s) and (resname %s)' % (sel,segid,resname)
                    viewer = renumber_resid_by_resid_vmd(lsel,mol,viewer,start=start)
            else:
                lsel = '(%s) and (segid %s)' % (sel,segid)
                viewer = renumber_resid_by_resid_vmd(lsel,mol,viewer,start=start)
    else:                      
        resnames = set(mol.get('resname',sel=sel))
        for resname in resnames:
            lsel = '(%s) and (resname %s)' % (sel,resname)
            viewer = renumber_resid_by_resid_vmd(lsel,mol,viewer,start=start)       
    viewer.send('animate write pdb {%s} waitfor all top;exit' % tmpout.name)
    newmol = Molecule(tmpout.name)
    tmpout.close()
    return newmol

def renumber_resid_by_resid_vmd(sel,mol,viewer,start=1):
    resids = sorted(list(set(mol.get('resid',sel=sel))))
    resids = [str(i) for i in resids]
    viewer.send('proc renum_resid {molid} {set newresid %d; set resids {%s};' % (start,' '.join(resids)) + \
                'set asall [atomselect $molid [concat {(%s) and resid } $resids]];' % sel + \
                '$asall set user 1.00;' + \
                'foreach resid $resids {' + \
                'set as [atomselect $molid [concat {user 1.00 and (%s) and resid } $resid]];' % sel + \
                '$as set resid $newresid; $as set user 0.00; incr newresid}};'+'renum_resid $molid')
    return viewer

def ordered_unique(seq):
    seen = set()
    return [x for x in seq if not (x in seen or seen.add(x))]

def renumber_segments(inputmol,segids,prefix):
    sel = 'segid '+' '.join(segids)
    segids = ordered_unique(inputmol.get('segid',sel=sel))
    if prefix in segids:
        raise ValueError('segid %s already exists.' % prefix)
    
    mol = renumber_resid_vmd(inputmol,sel,by=2)
    # change first segid segment as it is properly renumbered already
    mol.set('segid',prefix,sel='segid '+segids[0])

    if len(segids) > 1:
        # initialize variables for second segid
        curr_segid = prefix
        # get last resid for first segid
        idx_curr_segid = mol.atomselect('segid '+curr_segid)
        prev_resid = len(set(mol.resid[idx_curr_segid]))
        k = 0
        for segid in segids[1:]:
            
            # get last current resid
            idx_segid = mol.atomselect('segid '+segid)
            curr_resid = len(set(mol.resid[idx_segid])) + prev_resid
            if curr_resid <= 9999:
                # join segments resuming the previous resid numbering
                mol = renumber_resid_vmd(mol,'segid '+segid,start=prev_resid+1,by=2)
                mol.segid[idx_segid] = curr_segid
                # get last resid of the current segid for the next loop iteration
                prev_resid = curr_resid
            else:

                # join segments resuming the previous resid numbering up to resid 9999
                sel1 = 'segid '+segid+' and resid <= '+str(9999-prev_resid)
                mol = renumber_resid_vmd(mol,sel1,start=prev_resid+1,by=2)
                mol.set('segid',curr_segid,sel=sel1)
                # define next new segment with resids > 9999
                k +=1
                curr_segid = prefix+str(k)
                if curr_segid in segids:
                    raise ValueError('segid %s already exists.' % curr_segid)
                # resid <= 9999 still preserve the old segid
                idx_curr_segid = mol.atomselect('segid '+segid)
                mol.segid[idx_curr_segid] = curr_segid
                # get last resid of the current segid for the next loop iteration
                mol = renumber_resid_vmd(mol,'segid '+curr_segid,by=2)
                prev_resid = len(set(mol.resid[idx_curr_segid]))
            
        if k > 0:
            if prefix+str(0) in segids:
                print('WARNING: segid %s already exists, using %s instead.' % (prefix,prefix+str(0)))
            else:
                mol.segid[mol.segid == prefix] = prefix+str(0)
        
    return mol

def renumber_resid_by_resid(sel,mol,ordered=False):
    resids = list(set(mol.get('resid',sel=sel)))
    if ordered:
        resids = sort(resids)
    newresid = 1
    for resid in resids:
        mol.set('resid',newresid,sel='(%s) and (resid %s)' % (sel,resid))
        newresid += 1
    return mol

def renumber_resid(mol,sel,by=3):
    
    # Long story short: by=1 -> by_resname; by=2 -> by_segid; by=3 -> by_segid and by_resname
    # WTF!!!!
    option_num = 2
    max_value = 2**option_num - 1
    if by > max_value:
        raise ValueError('Maximum value for "by" keyword is %d.' % max_value)
    if by < 1:
        raise ValueError('Minimum value for "by" keyword is "1".')
    bin_by = format(by,'0'+str(option_num)+'b')
    option_array = [bool(int(i)) for i in bin_by]
    by_segid = option_array[0]
    by_resname = option_array[1]
    
    if by_segid:
        segids = set(mol.get('segid',sel=sel))      
        for segid in segids:
            if by_resname:
                resnames = set(mol.get('resname',sel='(%s) and segid %s' % (sel,segid)))
                for resname in resnames:
                    lsel = '(%s) and (segid %s) and (resname %s)' % (sel,segid,resname)
                    mol = renumber_resid_by_resid(lsel,mol)
            else:
                lsel = '(%s) and (segid %s)' % (sel,segid)
                mol = renumber_resid_by_resid(lsel,mol)
    else:                      
        resnames = set(mol.get('resname',sel=sel))
        for resname in resnames:
            lsel = '(%s) and (resname %s)' % (sel,resname)
            mol = renumber_resid_by_resid(lsel,mol)
    return mol

def fix_and_prepare_input(inputmol,first='NTER',last='CTER'):
    """
    ISMAEL FUNCTION
    Establish homogeneus nomenclature for protein residue names and segments for the system.
    """
    
    mol = inputmol.copy()
    aa= 'ALA ARG ASN ASP CYS GLU GLN GLY HIS ILE LEU LYS MET PHE PRO SER THR TRP TYR VAL'
    mol.set('segid','WAT',sel='water')
    mol.set('resname','TIP3',sel='water')
    mol.set('chain','X',sel='resname TIP3')
    mol.set('name','OH2',sel='resname TIP3 and name OW')
    mol.set('name','H1',sel='resname TIP3 and name HW1')
    mol.set('name','H2',sel='resname TIP3 and name HW2')
    mol.remove('(protein or resname '+aa+') and name O1 O2')
    if first == 'NTER':
        mol.set('name','HT1',sel='(protein or resname '+aa+')and name H1')
        mol.set('name','HT2',sel='(protein or resname '+aa+') and name H2')
        mol.set('name','HT3',sel='(protein or resname '+aa+') and name H3')
    else:
        mol.remove('(protein or resname '+aa+')and name H1 H2 H3')
    if last in {'CTER','CNEU','CTP','CT1'}:
        mol.set('name','OT1',sel='(protein or resname '+aa+') and name O1')
        mol.set('name','OT2',sel='(protein or resname '+aa+') and name O2')
        #fix
        mol.remove('(protein or resname '+aa+') and name OT2')
    else:
        mol.set('name','O',sel='(protein or resname '+aa+') and name O1')
        mol.remove('(protein or resname '+aa+') and name O2')
    mol.set('name','HG1',sel='resname CYS and name HG')
    mol.set('name','HN',sel='resname HIS and name H')
    
    his_he_resids = mol.get('resid',sel='resname HIS and name HE2')
    his_he_chains = mol.get('chain',sel='resname HIS and name HE2')
    his_he_ids = set([':'.join((chain,str(resid))) for resid,chain in zip(his_he_resids,his_he_chains)])
    his_hd_resids = mol.get('resid',sel='resname HIS and name HD1')
    his_hd_chains = mol.get('chain',sel='resname HIS and name HD1')
    his_hd_ids = set([':'.join((chain,str(resid))) for resid,chain in zip(his_hd_resids,his_hd_chains)])
    hsd_ids = his_hd_ids.difference(his_he_ids)
    hse_ids = his_he_ids.difference(his_hd_ids)
    hsp_ids = his_hd_ids.intersection(his_he_ids)
    hsd_dict = dict()
    hse_dict = dict()
    hsp_dict = dict()
    
    for chain,resid in [ id1.split(':') for id1 in hsd_ids]:
        if chain not in hsd_dict:
            hsd_dict[chain] = []
        hsd_dict[chain].append(resid)
    for chain,resid in [ id1.split(':') for id1 in hse_ids]:
        if chain not in hse_dict:
            hse_dict[chain] = []
        hse_dict[chain].append(resid)
    for chain,resid in [ id1.split(':') for id1 in hsp_ids]:
        if chain not in hsp_dict:
            hsp_dict[chain] = []
        hsp_dict[chain].append(resid)
    for chain in hsd_dict:
        sel1 = 'resname HIS and resid '+' '.join(hsd_dict[chain])
        mol.set('resname','HSD',sel=sel1)
    for chain in hse_dict:
        sel1 = 'resname HIS and resid '+' '.join(hse_dict[chain])
        mol.set('resname','HSE',sel=sel1)
    for chain in hsp_dict:
        sel1 = 'resname HIS and resid '+' '.join(hsp_dict[chain])
        mol.set('resname','HSP',sel=sel1)
    mol = autoSegment(mol,sel='protein or resname '+aa)
    mol.set('segid','LIG',sel='not (protein or resname '+aa+') and not water and not ions')
    mol.set('chain','L',sel='segid LIG')
    mol.set('segid','ION',sel='ions')
    mol.set('chain','N',sel='segid ION')
    protsegids = set(mol.get('segid',sel='protein'))
    mol = renumber_resid(mol,'water',by=1)
    mol = renumber_resid(mol,'ions',by=1)
    mol = renumber_resid(mol,'segid LIG',by=2)
    return (mol,protsegids)
    #    for segid in protsegids:
    #        resids = set(mol.get('resid',sel='segid '+segid))
    #        for resid in resids:
    #            resname = mol.get('resname',sel='resid '+str(resid)+' and segid '+segid)[0]
    #            chain = mol.get('chain',sel='resid '+str(resid)+' and segid '+segid)[0]
    #            mol.set('segid',segid,sel='resname '+resname+' and chain '+chain+' and resid '+str(segid))

def prepare_system(mol_aligned, pdbcode, thickness = None, sod2x50 = False):
    """
    Assign protonation states using "proteinPrepare" function from HTMD, and 
    force protonation of ASP2x50 if required
    """
    
    # If required, force protonation of ASP2x50
    if not sod2x50:
        # Download GPCRdb structure's website, and extract residue table from it
        structure_data = requests.get('https://gpcrdb.org/structure/homology_models/'+pdbcode+'_refined').content
        soup = BeautifulSoup(structure_data, 'html.parser')
        table = soup.find('table', attrs={'id':'rotamers'})
        table_body = table.find('tbody')
        rows = table_body.find_all('tr')

        # Iterate trougth residue table to find 2.50 residue ID in this PDB file
        asp2x50 = None
        for row in rows:
            cols = row.find_all('td')
            cols = [ele.text.strip() for ele in cols]
            if cols[2] == "2.50x50":
                asp2x50 = cols[1]

        # Force residue protonation
        current2x50 = mol_aligned.get('resname', sel='resid '+asp2x50)[0]
        if current2x50 in {'ASP','GLU'}:
            mol_aligned = proteinPrepare(mol_aligned,
                                         pH = 0,
                                         holdSelection = 'not resid '+asp2x50
                                 )
        
    # Prepare protein (assign protonation states to residues)
    prepared_mol, table = proteinPrepare(mol_aligned,
                                  hydrophobicThickness=thickness,
                                  returnDetails = True,
                                 )

    return prepared_mol
    
def add_membrane(pdbmol,membranemol,protsegids,membrane_distance,coldist=1.3):
    # Corrections for rotational difusion
    prot = pdbmol.copy()
    protsel = 'segid '+' '.join(protsegids)
    prot.filter(protsel,_logger=False)
    r = minimalRotation(prot)
    M = rotationMatrix([0, 0, 1], r)
    pdbmol.rotateBy(M)
    pcoor = pdbmol.get('coords',sel=protsel)
    Mcoor = np.max(pcoor,axis=0)
    mcoor = np.min(pcoor,axis=0)
    # get the diagonal of XY of the protein if XY is a square 
    # which side is as long as the largest side (between X and Y) from the protein box  
    p_dim = [Mcoor[0] - mcoor[0],Mcoor[1] - mcoor[1]]
    maxXY = np.sqrt(p_dim[0]**2+p_dim[1]**2)
    minimum_box_size_x = maxXY+2
    minimum_box_size_y = minimum_box_size_x
    
    # get min max coor of the system
    minc = np.min(pdbmol.coords, axis=0).flatten()
    maxc = np.max(pdbmol.coords, axis=0).flatten()
    
    system_size_x = maxc[0] - minc[0]
    system_size_y = maxc[1] - minc[1]
    
    center_x = system_size_x/2 + minc[0]
    center_y = system_size_y/2 + minc[1]
    
    system_size = np.max([system_size_x,system_size_y])
    corr_system_size_x = np.max([minimum_box_size_x,system_size]) 
    corr_system_size_y = np.max([minimum_box_size_y,system_size])
    
    addmembdist = membrane_distance/2.0+np.max([coldist,1.5])+0.0
    
    xlim = [center_x-corr_system_size_x/2-addmembdist,center_x+corr_system_size_x/2+addmembdist]
    ylim = [center_y-corr_system_size_y/2-addmembdist,center_y+corr_system_size_y/2+addmembdist]
    
    memb = membranemol.copy()
    memb.remove('ions',_logger=False)
    memb2 = tileMembrane(memb, xlim[0], ylim[0], xlim[1], ylim[1])
    
    #from tileMembrane
    size = np.max(membranemol.get('coords', 'water'), axis=0) - np.min(membranemol.get('coords', 'water'), axis=0)
    xreps = int(np.ceil((xlim[1] - xlim[0]) / size[0]))
    yreps = int(np.ceil((ylim[1] - ylim[0]) / size[1]))
    
    membtmp_segids = ordered_unique(memb2.get('segid'))
    k=0
    for segid in membtmp_segids:
    #    memb2.set('segid','M'+str(k),sel='segid '+segid+' and not waters')
         memb2.set('segid','W'+str(k),sel='segid '+segid+' and waters')
         k += 1
            
    memb2 = renumber_segments(memb2,set(memb2.get('segid',sel='waters')),'MW')
    memb2 = renumber_segments(memb2,set(memb2.get('segid',sel='not waters')),'MEM')
    membrane_resnames = set(memb2.get('resname'))
    membrane_segids = set(memb2.get('segid'))
    
    mcenter = np.mean(memb2.get('coords',sel='segid MEM'),axis=0)
    memb2.moveBy(-mcenter)

    memb2, num = removeLipidsInProtein(pdbmol, memb2,lipidsel='lipids or waters')
    
    mol = pdbmol.copy()
    mol.append(memb2, collisions=True,coldist=coldist)
    
    return (mol, membrane_resnames,membrane_segids,xreps,yreps)

def solvate_pdbmol(mol,membrane_segids,water_thickness,water_margin,buffer=2.4,coldist=1.3,prefix='W'):
    waterbox = mol.get('coords','(waters or ions) and segid '+' '.join(membrane_segids))
    mwaterbox = np.min(waterbox, axis=0)
    Mwaterbox = np.max(waterbox, axis=0)
    coo = mol.get('coords','not (waters or ions)')
    mcoo = np.min(coo, axis=0)
    Mcoo = np.max(coo, axis=0)
    cooall = mol.get('coords','all')
    mcooall = np.min(coo, axis=0)
    Mcooall = np.max(coo, axis=0)
    #top layer
    M1z = Mcoo[2] + water_thickness/2. + np.max((coldist,buffer)) - buffer
    m1z = Mwaterbox[2] - water_margin
    M1 = [Mwaterbox[0],Mwaterbox[1],M1z]
    m1 = [mwaterbox[0],mwaterbox[1],m1z]
    print("wataerbox Max and min: ", Mwaterbox, mwaterbox)
    
    #bottom layer
    M2z = mwaterbox[2] + water_margin
    m2z = mcoo[2] - water_thickness/2.- np.max((coldist,buffer)) + buffer
    M2 = [Mwaterbox[0],Mwaterbox[1],M2z]
    m2 = [mwaterbox[0],mwaterbox[1],m2z]

    smol = solvate(mol, minmax=np.vstack((m2,M1)),prefix=prefix,buffer=buffer)

    smol, num_remove = removeAtomsInHull(smol, smol, 'name CA', 'segid "'+prefix+'[0-9]+"')
    #wtsegids = set(smol.get('segid',sel='segid "%s.*"'% prefix))
    #for segid in wtsegids:
        #smol.remove('segid %s and same resid as ( z > %g and z < %g)' % (segid,M2[2],m1[2]),_logger=False)

    return smol

def add_dummy_atom(inputmol,property_dict):
    for prop in property_dict:
        dummymol.set(prop,property_dict[prop])
    inputmol.append(dummymol,coldist=None)
    return inputmol
def add_center_dummy_atom(inputmol,coords,property_dict):
    center = np.mean(coords,axis=0)
    property_dict['coords'] = center
    mol = add_dummy_atom(inputmol,property_dict)
    return mol
def remove_aromatic_insertions(inputmol,protsegids,coldist=1.5,outpdb=None):
    mol = inputmol.copy()
    atoms_data = [['TRP','CG CD1 CE1 NE1 CE2 CD2',5,'1'],
                 ['TRP','CD2 CE2 CZ2 CH2 CZ3 CE3',6,'2'],
                 ['PRO','N CA CB CG CD',5,''],
                 ['HIS HSD HSE HSP HID HIE HIP',
                  'CG CD1 CE1 CZ CE2 CD2 ND1 NE2',5,''],
                 ['PHE TYR TYM',
                  'CG CD1 CE1 CZ CE2 CD2',6,'']]
    beta_backup = np.copy(mol.beta)
    mol.set('beta',sequenceID((mol.resid, mol.insertion, mol.segid)))    
    
    for atom_data in atoms_data:
        atom_step = atom_data[2]
        suffix = atom_data[3]
        sel = 'resname %s and name %s' % (atom_data[0],atom_data[1])
        idxs = mol.get('index',sel=sel)
        resnames = mol.resname[idxs]
        resids = mol.resid[idxs]
        segids = mol.segid[idxs]
        coords = mol.coords[idxs,:,0]
        atom_num = len(idxs)
        if atom_num % atom_step != 0:
            raise ValueError('Missing atoms.')
        for i in range(0,atom_num,atom_step):
            property_dict = {'resname':resnames[i]+suffix,'resid':resids[i],'segid':segids[i]}
            mol = add_center_dummy_atom(mol,coords[i:i+atom_step],property_dict)
            
    if outpdb:
        mol.write(outpdb)
    var_list = tuple([coldist]+[dummy_sel for i in range(0,3)])
    
    dummy_atoms_idxs = mol.get('index',sel=dummy_sel)
    dummy_atoms_resid = mol.resid[dummy_atoms_idxs]
    dummy_atoms_segid = mol.segid[dummy_atoms_idxs]
    removed_indexes = []
    for idx,resid,segid in zip(dummy_atoms_idxs,dummy_atoms_resid,dummy_atoms_segid):
        r_idx1 = mol.get('index', sel='not (%s) and same beta as ((exwithin %g of (index %d)) and not (resid %d and segid %s))'  % (dummy_sel,coldist,idx,resid,segid))
        removed_indexes = removed_indexes + r_idx1.tolist()
        
    if len(removed_indexes) > 0:
        removed_indexes_str = ' '.join(str(x) for x in removed_indexes)
        mol.remove('index '+removed_indexes_str)
    mol.remove(dummy_sel,_logger=False)
    inv_idx1 = np.setdiff1d(np.arange(len(beta_backup)), np.array(removed_indexes), assume_unique=True)
    mol.beta = beta_backup[inv_idx1]
        
    print('WARNING: removed '+str(len(removed_indexes))+' atoms within '+str(coldist)+' of a protein aromatic ring')
    
    return (mol,removed_indexes)

def define_equilibration(const_sel):
    simtime = 40
    restr = AtomRestraint(const_sel, 2, [(0,"0"),(1,"%dns" % int(simtime*0.5)),(0,"%dns" % int(simtime*0.75))], "xyz")
    md = Equilibration()
    md.runtime = simtime
    md.timeunits = 'ns'
    md.temperature = 310
    md.nvtsteps = 0
    md.acemd.barostatconstratio = 'on'
    md.acemd.minimize = 5000
    md.acemd.restart = 'off'
    md.acemd.timestep = 2
    md.restraints = restr
    return md

def define_production():
    md = Production()
    md.runtime = 500
    md.timeunits = 'ns'
    md.temperature = 310
    md.acemd.timestep = 4
    md.acemd.barostatconstratio = 'on'
    md.acemd.restart = 'off'
    md.acemd.bincoordinates = 'output.coor'
    md.acemd.extendedsystem  = 'output.xsc'
    md.acemd.binvelocities = 'output.vel'
    return md

In [4]:
#################################################
## Part 1: Download data and prepare dictionaries
#################################################

if not bool(pdb_set):
    #Get not yet simulated PDB codes from GPCRdb
    pdb_set = get_GPCRdb_nonsimulated(gpcrdb_data)

# Download and store structures from GPCRdb
pdb_set = download_GPCRdb_structures(pdb_set, basepath)

#Create or moidfy the ligands dictionary
(ligandsdict, ligandsset) = ligand_dictionary(pdb_set, ligandsdict_path)

# Extract ligand structures from system
extract_ligands(ligandsdict, basepath)

# Get topology-parameter files for ligandsf
get_lig_toppar(ligandsset, basepath, username, password)


Structure for 5UIW already present. Skipping...
Structure for 6AKX already present. Skipping...
could not download 5V56 refined structure. Skipping...
Structure for 5TE3 already present. Skipping...
Structure for 6LI0 already present. Skipping...
Structure for 6D32 already present. Skipping...
Structure for 6TOS already present. Skipping...
Structure for 6IIU already present. Skipping...
could not download 6KPC refined structure. Skipping...
Structure for 5XSZ already present. Skipping...
Structure for 6D27 already present. Skipping...
Structure for 5TVN already present. Skipping...
Structure for 5WIV already present. Skipping...
Structure for 6A93 already present. Skipping...
Structure for 5X33 already present. Skipping...
Structure for 6ME2 already present. Skipping...
Structure for 6K1Q already present. Skipping...
Structure for 6LI1 already present. Skipping...
Structure for 5WIU already present. Skipping...
Structure for 5VBL already present. Skipping...
Structure for 6LI2 already

could not download 5MZP refined structure. Skipping...
Structure for 5O9H already present. Skipping...
Structure for 6PRZ already present. Skipping...
Structure for 6PS4 already present. Skipping...
Structure for 6TO7 already present. Skipping...
Structure for 6RZ9 already present. Skipping...
Structure for 6IGL already present. Skipping...
Structure for 6ME8 already present. Skipping...
Structure for 2YCZ already present. Skipping...
Structure for 6AQF already present. Skipping...
Structure for 5TZR already present. Skipping...
could not download 1GZM refined structure. Skipping...
Structure for 5UNG already present. Skipping...
Structure for 4EJ4 already present. Skipping...
Structure for 6FFI already present. Skipping...
Structure for 6KK7 already present. Skipping...
Structure for 6PS8 already present. Skipping...
Structure for 6TP4 already present. Skipping...
Structure for 6TOT already present. Skipping...
Structure for 4O9R already present. Skipping...
Structure for 6JZH already

toppar for ligand QNB already exists. Skipping...
Getting toppar file for ligand DGW (110/247)
toppar for ligand DGW already exists. Skipping...
Getting toppar file for ligand 8VS (111/247)
toppar for ligand 8VS already exists. Skipping...
Getting toppar file for ligand SUV (112/247)
toppar for ligand SUV already exists. Skipping...
Getting toppar file for ligand MLI (113/247)
toppar for ligand MLI already exists. Skipping...
Getting toppar file for ligand 2CV (114/247)
toppar for ligand 2CV already exists. Skipping...
Getting toppar file for ligand JDC (115/247)
toppar for ligand JDC already exists. Skipping...
Getting toppar file for ligand 2YB (116/247)
toppar for ligand 2YB already exists. Skipping...
Getting toppar file for ligand P6G (117/247)
toppar for ligand P6G already exists. Skipping...
Getting toppar file for ligand NO3 (118/247)
toppar for ligand NO3 already exists. Skipping...
Getting toppar file for ligand 68H (119/247)
toppar for ligand 68H already exists. Skipping...


toppar for ligand SIN already exists. Skipping...
Getting toppar file for ligand NRK (218/247)
toppar for ligand NRK already exists. Skipping...
Getting toppar file for ligand 9XW (219/247)
toppar for ligand 9XW already exists. Skipping...
Getting toppar file for ligand ZAL (220/247)
toppar for ligand ZAL already exists. Skipping...
Getting toppar file for ligand ZD7 (221/247)
toppar for ligand ZD7 already exists. Skipping...
Getting toppar file for ligand BGC (222/247)
toppar for ligand BGC already exists. Skipping...
Getting toppar file for ligand NH2 (223/247)
toppar for ligand NH2 already exists. Skipping...
Getting toppar file for ligand PE5 (224/247)
toppar for ligand PE5 already exists. Skipping...
Getting toppar file for ligand GMJ (225/247)
toppar for ligand GMJ already exists. Skipping...
Getting toppar file for ligand ON9 (226/247)
toppar for ligand ON9 already exists. Skipping...
Getting toppar file for ligand 8K8 (227/247)
toppar for ligand 8K8 already exists. Skipping...


In [ ]:
###########################
## Part 2: Build the models 
###########################
  
# Iterate by GPCRdb structures to simulate
pdbs_number = len(pdb_set)
i = 0
for pdbcode in pdb_set:
#for pdbcode in ['5YQZ']:
    try:
        #Starting simulation
        start_time = time.time()        
        i += 1
        print('\nstart building model for receptor %s (%d/%d)' % (pdbcode, i, pdbs_number))
        # Skip if there is already a model build for this
        if os.path.exists(resultspath+'build/'+pdbcode+'/structure.pdb'):
            #print('Build model for '+pdbcode+' already exists. Skipping...')
            continue

        # Add internal waters to GPCRdb structures, using HomolWat
        simdir = basepath + 'data_sim/'+pdbcode+'/'
        sod2x50 = internal_waters(simdir, pdbcode, gpcrdb_data)

        # Load GPCRdb and OPM versions of GPCR with pdbcode
        gpcrdb_mol = Molecule(glob(simdir + '*_HW.pdb'))
        thickness,opm_mol = get_opm(pdbcode)
        
        # Remove unnecessary ligand molecules: mostly crystalization detergents, quelants, buffers,
        # or post-traductional glicosilations
        gpcrdb_mol.remove('resname '+' '.join(detergent_blacklist))
        gpcrdb_mol.remove('resname '+' '.join(glucids_blacklist))
        
        # Ismael's function to add labels (segid) for 'ligand' and 'protein' parts of the system
        #And many things more I do not really understand
        gpcrdb_mol_fixed,receptor_segids_gpcrdb = fix_and_prepare_input(gpcrdb_mol)
        opm_mol_fixed,receptor_segids_opm = fix_and_prepare_input(opm_mol)

        # Paths and previous variables
        modelname = pdbcode # Example name
        opm_modelname = pdbcode + '_opm'
        
        # Assigning new chain to protein segment of the protein to align (opm and gpcrdb)
        opm_receptorsel = 'segid '+' '.join(receptor_segids_opm)
        opm_mol_fixed.set('chain',new_pdb_chain,sel=opm_receptorsel)
        gpcrdb_receptorsel = 'segid '+' '.join(receptor_segids_gpcrdb)
        gpcrdb_mol_fixed.set('chain',new_pdb_chain,sel=gpcrdb_receptorsel)

        # Align structrues using sequences, and take first one
        alignment_results = sequenceStructureAlignment(gpcrdb_mol_fixed, opm_mol_fixed, maxalignments = 1)
        mol_aligned = alignment_results[0] 

        #Center to receptor XY
        center = np.mean(mol_aligned.get('coords',sel=gpcrdb_receptorsel),axis=0)
        mol_aligned.moveBy([-center[0],-center[1],0])

        # Prepare protein: asign titration states, flipping side chains of HIS, ASN and GLN; rotate some sidechains, optimize waters, etc.
        # Most of this is done with a HTMD function called proteinPrepare()
        prepared_mol = prepare_system(mol_aligned, pdbcode, thickness, sod2x50)
        
        #Add membrane
        print('Adding membrane...')
        membranemol = Molecule(membranepdb)
        mol_membraned, membrane_resnames, membrane_segids, xreps, yreps = add_membrane(mol_aligned, membranemol,receptor_segids_gpcrdb,membrane_distance)

        # Needed later for equilibration
        with open(simdir+"const_sel.txt",'w') as out: 
            const_sel = 'segid '+' '.join(receptor_segids_gpcrdb)+' and name C CA N O or not (segid ' + \
              ' '.join(receptor_segids_gpcrdb)+' or resname '+' '.join(membrane_resnames) + \
              ' or water or ions ) and noh or segid ION WAT and noh'
            out.write(const_sel)
            
        #Solvate
        print('Solvating...')
        mol_solvated = solvate_pdbmol(mol_membraned,membrane_segids,water_thickness,water_margin,buffer=buffer,coldist=coldist,prefix='WT')

        # Make list of Ligand stringfiles (Parameters and topology)
        streams = []
        for ligcode in ligandsdict[pdbcode]:
            streams.append(basepath + 'Ligands/'+ ligcode+ '/toppar.str')

        # Assignign terminology for cap atoms of protein chain, depending if it is the receptor protein or not
        caps_receptor = ['first ACE', 'last CT3']
        caps_not_receptor_protein = ['first NTER', 'last CTER']
        caps = { segid : caps_receptor for segid in receptor_segids_gpcrdb }

        #Pre-build model
        print('Pre-build...')
        prebuildmol = charmm.build(mol_solvated, 
                                   topo=topos, 
                                   param=params,
                                   stream=streams,
                                   caps=caps,
                                   outdir=resultspath+'/pre-build/'+modelname,
                                   ionize=False)

        # Save prebuild model topologies in files, and  store prebuild model in molecule object
        prebuild_psffile = prebuildmol.topoloc
        prebuild_pdbfile = os.path.splitext(prebuildmol.topoloc)[0]+'.pdb'
        prebuildmol = Molecule(prebuild_pdbfile)
        _recoverProtonations(prebuildmol)

        # Checking of aromatic insertions (takes quite a lot fo time)
        print('Checking aromatic insertions...')
        mol_removed,removed_indexes = remove_aromatic_insertions(mol_solvated,receptor_segids_gpcrdb, outpdb=resultspath+'/pre-build/'+modelname+'/aromatic_check.pdb')

        # Checking of water/lipid ratio
        lipid_num = len(set(mol_removed.get('resid',sel='segid '+membrane_lipid_segid)))
        solv_num = len(mol_removed.get('index',sel='resname TIP3 and name OH2'))
        if float(solv_num) / lipid_num < 35:
            raise ValueError('Water/lipid ratio lower than 35.')

        #Renumber residues
        print('Renumbering...')
        mol_renumbered = renumber_resid_vmd(mol_removed,'segid '+' '.join(membrane_segids),by=2)

        # Ionizing system
        print('Ionizing...')
        molbuilt = charmm.build(mol_removed,
                                topo=topos,
                                stream=streams,                        
                                param=params,
                                outdir=resultspath+'/ionize/'+modelname,
                                saltconc=0.15,
                                caps=caps)
        build_psffile = molbuilt.topoloc
        build_pdbfile = os.path.splitext(molbuilt.topoloc)[0]+'.pdb'
        molbuilt = Molecule(build_pdbfile)
        _recoverProtonations(molbuilt)

        #Building system
        print('Building...')
        molbuilt = renumber_resid_vmd(molbuilt,'segid "WT.*" or segid I',by=2)
        molbuilt = charmm.build(molbuilt, 
                                topo=topos, 
                                stream=streams,                        
                                param=params,
                                outdir=resultspath+'/build/'+modelname,
                                caps=caps,ionize=False)

        print('End of %s after %s seconds\n' % (modelname, time.time() - start_time))
    except Exception as e:
        print("model "+pdbcode+" could not be build because ",e)


start building model for receptor 5UIW (1/181)

start building model for receptor 6AKX (2/181)
Structure 6AKX already has a watered version. Skipping...


2020-09-09 09:47:03,303 - htmd.molecule.molecule - INFO - Removed 0 atoms. 2586 atoms remaining in the molecule.
2020-09-09 09:47:03,347 - htmd.molecule.molecule - INFO - Removed 0 atoms. 2586 atoms remaining in the molecule.
2020-09-09 09:47:03,646 - htmd.molecule.molecule - INFO - Removed 0 atoms. 2586 atoms remaining in the molecule.
2020-09-09 09:47:03,886 - htmd.molecule.molecule - INFO - Removed 0 atoms. 2586 atoms remaining in the molecule.
2020-09-09 09:47:08,778 - htmd.molecule.molecule - INFO - Removed 0 atoms. 2443 atoms remaining in the molecule.
2020-09-09 09:47:09,007 - htmd.molecule.molecule - INFO - Removed 0 atoms. 2443 atoms remaining in the molecule.
2020-09-09 09:47:09,410 - htmd.builder.builder - INFO - Created segment P0 between resid 19 and 223.
2020-09-09 09:47:09,412 - htmd.builder.builder - INFO - Created segment P1 between resid 227 and 313.
2020-09-09 09:47:10,264 - moleculekit.tools.sequencestructuralalignment - INFO - No segment was specified by the user f

Adding membrane...


2020-09-09 09:47:32,909 - htmd.builder.builder - INFO - Replicating Membrane 3x3

Replicating Membrane:  44%|████▍     | 4/9 [00:04<00:05,  1.02s/it]
2020-09-09 09:48:59,990 - htmd.molecule.molecule - INFO - Removed 2177 atoms. 38309 atoms remaining in the molecule.
2020-09-09 09:49:02,654 - htmd.molecule.molecule - INFO - Removed 107 residues from appended Molecule due to collisions.


Solvating...
wataerbox Max and min:  [49.473194 48.229332 28.214697] [-44.138805 -44.292667 -27.580303]


...

2020-09-09 12:55:04,853 - htmd.builder.builder - INFO - Replicating Membrane 3x3

Replicating Membrane:  67%|██████▋   | 6/9 [00:04<00:02,  1.27it/s]
2020-09-09 12:56:54,869 - htmd.molecule.molecule - INFO - Removed 2374 atoms. 40988 atoms remaining in the molecule.
2020-09-09 12:56:57,856 - htmd.molecule.molecule - INFO - Removed 78 residues from appended Molecule due to collisions.


Solvating...
wataerbox Max and min:  [48.49004  50.03344  28.174822] [-46.143963 -46.31056  -27.620178]


2020-09-09 12:56:59,393 - htmd.builder.solvate - INFO - Using water pdb file at: /soft/EB_repo/devel/programs/foss/2016b/Anaconda3/5.0.1/lib/python3.6/site-packages/htmd/builder/wat.pdb
2020-09-09 12:57:01,313 - htmd.builder.solvate - INFO - Replicating 8 water segments, 2 by 2 by 2

Solvating: 100%|██████████| 8/8 [00:21<00:00,  2.63s/it]
2020-09-09 12:57:26,200 - htmd.builder.solvate - INFO - 12562 water molecules were added to the system.
2020-09-09 12:57:41,914 - htmd.molecule.molecule - INFO - Removed 777 atoms. 75826 atoms remaining in the molecule.


Pre-build...


2020-09-09 12:57:43,804 - htmd.builder.charmm - INFO - Writing out segments.


In [9]:
#########################
## Part 3: Equillibration
#########################

def define_equilibration(const_sel):
    simtime = 40
    restr = AtomRestraint(const_sel, 2, [(0,"0"),(1,"%dns" % int(simtime*0.5)),(0,"%dns" % int(simtime*0.75))], "xyz")
    md = Equilibration()
    md.runtime = simtime
    md.timeunits = 'ns'
    md.temperature = 310
    md.nvtsteps = 0
    md.acemd.barostatconstratio = 'on'
    md.acemd.minimize = 5000
    #md.acemd.minimize = str(5000)
    md.acemd.restart = 'off'
    md.acemd.timestep = 2
    md.restraints = restr
    md._version = 3
    return md


#Launch equilibration
try:
    sqs
except NameError:
    sqs = {}

#for pdbcode in pdb_set:
for pdbcode in ['4EJ4']:
    modelname = pdbcode
    pdbfile = '%s/build/%s/structure.pdb' % (resultspath, pdbcode)
    if modelname in sqs:
        print('Skipping '+modelname+': it has already been submitted.')
        #continue

    # Preparing scripts to run equillibration
    equildir = resultspath+'/equil/'+modelname+'/'
    if not os.path.exists(equildir):
        os.makedirs(equildir)

    # Taking vmd selection line
    with open(basepath+'data_sim/'+pdbcode+'/const_sel.txt', 'r') as outfile:
        const_sel = outfile.readlines()[0]
    
    md = define_equilibration(const_sel)
    md.write(resultspath+'build/'+modelname,equildir)

    #Substitute run.sh generated by HTMD by a different one, adapted to the specified path of ACEMD
    with open(equildir + 'run.sh', 'w') as f:
        f.write('#!/bin/bash\n%s >log.txt 2>&1' % acemd_path)
        
    sq = SlurmQueue()
    sq.envvars = acemd_license
    sq.jobname = 'eql_'+pdbcode
    sq.datadir = None
    sq.partition = 'gpcr_gpu'
    sq.ngpu = 1
    sq.ncpu = 1
    sq.nodelist = 'bombur'
    
    #sq.exclude = 'excluded_node'
    
    # directory to copy input and store output of equilibration (initial working directory for run_equil.sh).
    # equildir directory has to be in the computation server, or in a shared folder for the computation folder.
    equildir = resultspath + '/equil/'+modelname+'/'
    # copy equil folder in build to equildir
    #copytree(resultspath+'/build/'+modelname+'/equil',equildir)
    sq.submit(equildir)
    sqs[modelname] = sq
sqs = {}

2020-09-01 08:52:03,139 - htmd.builder.builder - WARNING - Found cis peptide bond in 1 frames: [0] in the omega diheral "Angle of (SER 204 CA P0 P) (SER 204 C P0 P) (PRO 205 N P0 P) (PRO 205 CA P0 P) " with indexes [2602, 2609, 2611, 2615]
2020-09-01 08:52:03,140 - htmd.builder.builder - WARNING - Found cis peptide bond in 1 frames: [0] in the omega diheral "Angle of (THR 335 CA P0 P) (THR 335 C P0 P) (PRO 336 N P0 P) (PRO 336 CA P0 P) " with indexes [4790, 4800, 4802, 4806]
2020-09-01 08:52:03,534 - htmd.protocols.equilibration_v2 - WARNING - Lipids detected in input structure. We highly recommend setting useconstantratio=True for membrane simulations.
2020-09-01 08:52:03,534 - htmd.protocols.equilibration_v2 - INFO - Using user-provided restraints and ignoring constraints and fb_potential
2020-09-01 08:52:07,090 - jobqueues.slurmqueue - INFO - Queueing /gpcr/users/daranda/doctorat/GPCR_simulations/simulation_output//equil/4EJ4/


In [ ]:
# reset tracking for all
sqs = {}

In [47]:
# WARNING!!!: run me to KILL simulations that are still running
for modelname in sqs:
    sqs[modelname].stop()

In [ ]:
############ Equilibration commands and parameters

#run me to check how many simulations are still running
sum([sqs[modelname].inprogress() for modelname in sqs])    

In [10]:
#####################
## Part 4: Production
#####################

def define_production():
    md = Production()
    md.runtime = 500
    md.timeunits = 'ns'
    md.temperature = 310
    md.acemd.timestep = 4
    md.acemd.barostatconstratio = 'on'
    md.acemd.barostat = 'off'
    md.acemd.barostatpressure = 1.01325
    md.acemd.restart = 'off'
    md.acemd.bincoordinates = 'output.coor'
    md.acemd.extendedsystem  = 'output.xsc'
    md.acemd.binvelocities = 'output.vel'
    return md
sqs_p = {}

# Production protocol
md = define_production()

# If some model should be skipped, put its name here
modelname_skip = {}

# number of replicates (provisionaly one)
repnum = 1
# Create pdbid-to-productionObject dictionary if not exists yet
try:
    sqs_p
except NameError:
    sqs_p = {}

# For each PDB 
for pdbcode in pdb_set:
    
    # must match with equildir in equilibration launcher code and contain input and output of equilibration.
    modelname = pdbcode
    equildir = '%s/equil/%s/' % (resultspath, modelname)
    for rep in range(1,repnum+1):
        if modelname in modelname_skip:
            print('Skipping '+modelname+'_'+str(rep)+'.')
            continue
        if modelname+'_'+str(rep) in sqs_p:
            print('Skipping '+modelname+'_'+str(rep)+': it has already been submitted.')
            continue
        md = define_production()
        # directory copy output of equilibration to production input (initial working directory for run_prod.sh).
        proddir='%sproduction/%s/rep_%d/' % (resultspath, modelname, rep)
        md.write(equildir,proddir)
        
        # This replaces htmd generated run.sh. Input and output for simulations is set in bash_script_folder+'/run_equil.sh'.
        # Memo 7-9-1017: datadir = inputdir+'/../../data/'+modelname #defined in bash_script_folder+'/run_equil.sh
        #copy2(bash_script_folder+'/run_prod.sh',inputdir+'/run.sh')
        #copy2(bash_script_folder+'/copyback.sh',inputdir)
        
        sq = SlurmQueue()
        sq.envvars = acemd_license
        sq.jobname = modelname+'_pr'+str(rep)
        sq.datadir = None
        sq.partition = 'gpcr_gpu'
        sq.ngpu = 1
        sq.ncpu = 2
        sq.nodelist = ['dwalin', 'gimli']
        
        #Substitute run.sh generated by HTMD by a different one, adapted to the specified path of ACEMD
        with open(proddir + 'run.sh', 'w') as f:
            f.write('#!/bin/bash\n%s >log.txt 2>&1' % acemd_path)
        
        sqs_p[modelname+'_'+str(rep)] = sq
        sq.submit(proddir)


2020-09-01 13:03:50,736 - htmd.builder.builder - WARNING - Found cis peptide bond in 1 frames: [0] in the omega diheral "Angle of (SER 204 CA P0 P) (SER 204 C P0 P) (PRO 205 N P0 P) (PRO 205 CA P0 P) " with indexes [2602, 2609, 2611, 2615]
2020-09-01 13:03:50,737 - htmd.builder.builder - WARNING - Found cis peptide bond in 1 frames: [0] in the omega diheral "Angle of (THR 335 CA P0 P) (THR 335 C P0 P) (PRO 336 N P0 P) (PRO 336 CA P0 P) " with indexes [4790, 4800, 4802, 4806]
2020-09-01 13:03:51,148 - htmd.protocols.production_v6 - WARNING - Lipids detected in input structure. We highly recommend setting useconstantratio=True for membrane simulations.


RuntimeError: Could not find any of the files "output.coor" specified for command "bincoordinates" in path /gpcr/users/daranda/doctorat/GPCR_simulations/simulation_output//equil/4EJ4/

In [ ]:
# WARNING!!!: run me to KILL simulations that are still running
for modelname_rep in sqs_p:
    sqs_p[modelname_rep].stop()

In [13]:
# reset tracking for all
sqs_p = {}